In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '../../bar_movement/data/'

In [3]:
currency_pair = 'Eur_Usd'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_long = pd.read_csv(file_path + f'Oanda_{currency_pair}_M30_2021-2022.csv')
df_long.Date = pd.to_datetime(df_long.Date)
df_long.reset_index(drop=True, inplace=True)

In [5]:
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth

def add_fractal(df, i, look_back=3):
    if look_back <= i < df.shape[0] - look_back:
        lows = []
        highs = []

        for j in range(1, look_back + 1):
            prev_mid_low, prev_mid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
            future_mid_low, future_mid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

            lows.append(float(prev_mid_low))
            lows.append(float(future_mid_low))
            highs.append(float(prev_mid_high))
            highs.append(float(future_mid_high))

        mid_low, mid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

        if float(mid_low) < min(lows):
            return float(mid_low), np.nan

        elif float(mid_high) > max(highs):
            return np.nan, float(mid_high)

        else:
            return np.nan, np.nan

    else:
        return np.nan, np.nan

df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

tups = [add_fractal(df_long, i) for i in range(df_long.shape[0])]
supports, resistances = [tup[0] for tup in tups], [tup[1] for tup in tups]
df_long['support'], df_long['resistance'] = supports, resistances
df_long = df_long.fillna(method='ffill')

df_long.dropna(inplace=True)
df_long.reset_index(drop=True, inplace=True)

df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema200,macd,macdsignal,adx
0,2021-04-01 07:10:00,1.17239,1.17334,1.17239,1.17316,1.17252,1.17348,1.17252,1.17330,1.17246,1.17341,1.17246,1.17323,662,1.172286,0.000138,0.000090,38.394956
1,2021-04-01 07:15:00,1.17315,1.17321,1.17274,1.17297,1.17329,1.17334,1.17287,1.17310,1.17322,1.17328,1.17281,1.17304,490,1.172337,0.000154,0.000103,46.548073
2,2021-04-01 07:20:00,1.17298,1.17362,1.17263,1.17362,1.17311,1.17375,1.17276,1.17375,1.17304,1.17368,1.17270,1.17368,548,1.172422,0.000203,0.000124,49.272128
3,2021-04-01 07:25:00,1.17364,1.17420,1.17352,1.17414,1.17377,1.17433,1.17366,1.17427,1.17370,1.17426,1.17359,1.17420,698,1.172530,0.000272,0.000154,50.893711
4,2021-04-01 07:30:00,1.17412,1.17425,1.17381,1.17394,1.17425,1.17438,1.17391,1.17405,1.17418,1.17432,1.17386,1.17400,398,1.172614,0.000308,0.000185,53.478762


In [6]:
df_long.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,support,resistance
0,2021-04-01 09:00:00,1.17270,1.17414,1.17237,1.17402,1.17283,1.17426,1.17249,1.17414,1.17276,1.17420,1.17243,1.17408,2220,1.17243,1.17446
1,2021-04-01 09:30:00,1.17401,1.17438,1.17365,1.17394,1.17413,1.17450,1.17378,1.17405,1.17407,1.17444,1.17372,1.17400,1325,1.17243,1.17446
2,2021-04-01 10:00:00,1.17395,1.17499,1.17370,1.17465,1.17407,1.17511,1.17383,1.17477,1.17401,1.17505,1.17376,1.17471,1619,1.17243,1.17505
3,2021-04-01 10:30:00,1.17463,1.17474,1.17346,1.17364,1.17475,1.17485,1.17358,1.17376,1.17469,1.17480,1.17352,1.17370,1586,1.17243,1.17505
4,2021-04-01 11:00:00,1.17365,1.17423,1.17324,1.17418,1.17377,1.17436,1.17336,1.17431,1.17371,1.17430,1.17330,1.17424,1812,1.17330,1.17505


In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100

In [8]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, level_percentage, adx_cutoff):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    lookback = 12

    for i in range(lookback, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]

        filtered = df_long[df_long.Date >= curr_date]

        if len(filtered) == 0:
            break

        curr_long = df_long[df_long.Date <= curr_date]

        if len(curr_long) < 2:
            continue
          
        support, resistance = curr_long.loc[curr_long.index[-2], ['support', 'resistance']]
        sr_diff = abs(resistance - support)

        spread = abs(curr_ask_open - curr_bid_open)

        mid_highs = list(df.loc[df.index[i - lookback:i], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - lookback:i], 'Mid_Low'])
        highest_high, lowest_low = max(mid_highs), min(mid_lows)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        ema, adx, mid_close1, macd1, macdsignal1 = df.loc[df.index[i - 1], ['ema200', 'adx', 'Mid_Close', 'macd', 'macdsignal']]

        close_to_support = (abs(mid_close1 - support) / sr_diff) <= level_percentage and mid_close1 >= support
        close_to_resistance = (abs(resistance - mid_close1) / sr_diff) <= level_percentage and mid_close1 <= resistance

        macd_vals = [macd2, macdsignal2, macd1, macdsignal1, 0]
        macd_buy_signal = macd2 < macdsignal2 and macd1 > macdsignal1 and max(macd_vals) == 0
        macd_sell_signal = macd2 > macdsignal2 and macd1 < macdsignal1 and min(macd_vals) == 0

        ema_buy_signal = mid_close1 > ema
        ema_sell_signal = mid_close1 < ema

        adx_signal = adx >= adx_cutoff if adx_cutoff is not None else True

        buy_signal = macd_buy_signal and ema_buy_signal and close_to_support and adx_signal
        sell_signal = macd_sell_signal and ema_sell_signal and close_to_resistance and adx_signal

        if trade is None:
            if buy_signal:
                open_price = float(curr_ask_open)

                stop_loss = lowest_low
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(curr_bid_open)
                
                stop_loss = highest_high
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)


        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [9]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.0, 1.5, 2.0]
spread_cutoffs = [0.10, 0.20]
level_percentages = [0.01, 0.05, 0.10, 0.15]
adx_cutoffs = [None, 25, 30]  # ADX is between 0 and 100; usually ADX > 25 means the market is moving fairly well

all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for level_percentage in level_percentages:
            for adx_cutoff in adx_cutoffs:
                all_combos.append((risk_reward_ratio, spread_val, level_percentage, adx_cutoff))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_level_percentage = None
best_adx_cutoff = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, level_percentage, adx_cutoff in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, level_percentage, adx_cutoff)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'level_percentage': level_percentage, 'adx_cutoff': adx_cutoff})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_level_percentage = level_percentage
        best_adx_cutoff = adx_cutoff

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 72

-49.99887999999888
Num buys: 1
Num sells: 0
Num trades: 1
Num wins: 0
Num losses: 1
Win streak: 0
Loss streak: 1
Avg pips risked: 0.0015199999999999658
Remaining runs: 71
Best reward so far: -49.99887999999888

-49.99887999999888
Num buys: 1
Num sells: 0
Num trades: 1
Num wins: 0
Num losses: 1
Win streak: 0
Loss streak: 1
Avg pips risked: 0.0015199999999999658
Remaining runs: 70
Best reward so far: -49.99887999999888

0
Num buys: 0
Num sells: 0
Num trades: 0
Num wins: 0
Num losses: 0
Win streak: 0
Loss streak: 0
Remaining runs: 69
Best reward so far: 0

-149.9980600000043
Num buys: 2
Num sells: 1
Num trades: 3
Num wins: 0
Num losses: 3
Win streak: 0
Loss streak: 3
Avg pips risked: 0.0010766666666666886
Remaining runs: 68
Best reward so far: 0

-49.999679999998605
Num buys: 1
Num sells: 0
Num trades: 1
Num wins: 0
Num losses: 1
Win streak: 0
Loss streak: 1
Avg pips risked: 0.0008099999999999774
Remaining runs: 67
Best reward so far: 0

-149.9980600000043
Num buys: 2
Num se

In [22]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best level percentage: ' + str(best_level_percentage))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 49.99800000000019
Best risk/reward ratio: 1.0
Best spread: 0.1
Best level percentage: 0.1
Best adx cutoff: None
-----------------------
Top results:
{'reward': 49, 'ratio': 1.0, 'spread': 0.1, 'level_percentage': 0.1, 'adx_cutoff': None}
{'reward': 0, 'ratio': 1.0, 'spread': 0.1, 'level_percentage': 0.1, 'adx_cutoff': 30}
{'reward': 0, 'ratio': 2.0, 'spread': 0.1, 'level_percentage': 0.01, 'adx_cutoff': 30}
{'reward': 0, 'ratio': 1.5, 'spread': 0.1, 'level_percentage': 0.1, 'adx_cutoff': 30}
{'reward': 0, 'ratio': 1.0, 'spread': 0.1, 'level_percentage': 0.05, 'adx_cutoff': 25}
{'reward': 0, 'ratio': 1.5, 'spread': 0.1, 'level_percentage': 0.01, 'adx_cutoff': 30}
{'reward': 0, 'ratio': 1.0, 'spread': 0.1, 'level_percentage': 0.01, 'adx_cutoff': 25}
{'reward': 49, 'ratio': 1.0, 'spread': 0.1, 'level_percentage': 0.15, 'adx_cutoff': None}
{'reward': 49, 'ratio': 1.0, 'spread': 0.1, 'level_percentage': 0.05, 'adx_cutoff': None}
{'reward'